In [4]:
#-*- coding: UTF-8 -*-
import urllib
import pprint
import time
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [18]:
#網頁的編碼是GBK(中國區域編碼)


for page in range(0,10):
    for sub_page in range(0,10):
        novel_URL = "https://www.wenku8.net/novel/" +  str(page) + "/" + str(sub_page) + "/index.htm"
        try:
            response = urllib.request.urlopen(novel_URL)
            html = response.read()
            print("網址為 {0}".format(novel_URL))
            soup = BeautifulSoup(html, "html5lib")
            print("現在的目錄為 {0} 子目錄為 {1}".format(page, sub_page))
            print("===" * 10)
            print(soup.find(id = "title"))
            print(soup.find(id = "info"))
            for table in soup.find_all("td", {"class":"vcss"}):
                print(table)
            print("暫停1秒鐘")
            time.sleep(1)
        except HTTPError:
            print("網址 {0} 沒有小說".format(novel_URL))
            print("暫停1秒鐘")
            time.sleep(1)
            continue

網址 https://www.wenku8.net/novel/0/0/index.htm 沒有小說
暫停1秒鐘
網址為 https://www.wenku8.net/novel/0/1/index.htm
現在的目錄為 0 子目錄為 1
<div id="title">文学少女</div>
<div id="info">作者：野村美月</div>
<td class="vcss" colspan="4">第一卷 渴望死亡的小丑</td>
<td class="vcss" colspan="4">第二卷 渴求真爱的幽灵</td>
<td class="vcss" colspan="4">第三卷 沉陷过往的愚者</td>
<td class="vcss" colspan="4">第四卷 背负污名的天使</td>
<td class="vcss" colspan="4">第五卷 悲怆恸哭的巡礼者</td>
<td class="vcss" colspan="4">第六卷 孕育月花的水妖</td>
<td class="vcss" colspan="4">第七卷 惠临神明的作家（上）</td>
<td class="vcss" colspan="4">第八卷 惠临神明的作家（下）</td>
<td class="vcss" colspan="4">番外篇 今天的点心</td>
<td class="vcss" colspan="4">文学少女的秘密书架</td>
<td class="vcss" colspan="4">外传一 见习生的初恋</td>
<td class="vcss" colspan="4">“文学少女”登上石像怪与笨蛋的阶梯</td>
<td class="vcss" colspan="4">恋爱插话集第一弹</td>
<td class="vcss" colspan="4">恋爱插话集第二弹</td>
<td class="vcss" colspan="4">外传二 见习生的伤心</td>
<td class="vcss" colspan="4">恋爱插话集第三弹</td>
<td class="vcss" colspan="4">外传三 见习生的毕业</td>
<td class="vcss" colspan="4">恋爱插话集第四弹</td>
<t

KeyboardInterrupt: 